In [1]:
# train_save_models.py
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import TimeSeriesSplit
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
import joblib
from datetime import datetime
import json
from sklearn.isotonic import IsotonicRegression
from sklearn.metrics import brier_score_loss, log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
import glob

In [2]:
FEATURES_DIR = "data/features"
INPUT_FILE = os.path.join(FEATURES_DIR, "BTC-USD_daily_labeled.parquet")
EXPERIMENTS_DIR = "experiments"
SEQ_LEN = 20
BATCH_SIZE = 32
EPOCHS = 30
N_SPLITS = 7  # for OOF predictions

In [3]:
def build_lstm(seq_len, num_features, config="medium"):
    """
    Build LSTM model based on config size: 'small', 'medium', 'large'
    """
    if config == "small":
        lstm_units = [64]
        dense_units = [32]
        dropout_rate = 0.3

    elif config == "medium":
        lstm_units = [64, 32]
        dense_units = [32]
        dropout_rate = 0.3

    elif config == "large":
        lstm_units = [128, 64]
        dense_units = [64, 32]
        dropout_rate = 0.4

    # Build Sequential Model
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=(seq_len, num_features)))

    # Add LSTM layers
    for i, units in enumerate(lstm_units):
        return_sequences = (i < len(lstm_units) - 1)  # True except last LSTM
        model.add(tf.keras.layers.LSTM(units, return_sequences=return_sequences))
        model.add(tf.keras.layers.Dropout(dropout_rate))

    # Add Dense layers
    for units in dense_units:
        model.add(tf.keras.layers.Dense(units, activation="relu"))
        model.add(tf.keras.layers.Dropout(dropout_rate))

    # Output layer
    model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

    # Compile
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )
    return model

def build_gru(seq_len, num_features, config="medium"):
    """
    Build GRU model based on config size: 'small', 'medium', 'large'
    """
    if config == "small":
        gru_units = [64]
        dense_units = [32]
        dropout_rate = 0.3

    elif config == "medium":
        gru_units = [64, 32]
        dense_units = [32]
        dropout_rate = 0.3

    elif config == "large":
        gru_units = [128, 64]
        dense_units = [64, 32]
        dropout_rate = 0.4

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=(seq_len, num_features)))

    # GRU layers
    for i, units in enumerate(gru_units):
        return_sequences = (i < len(gru_units) - 1)
        model.add(tf.keras.layers.GRU(units, return_sequences=return_sequences))
        model.add(tf.keras.layers.Dropout(dropout_rate))

    # Dense layers
    for units in dense_units:
        model.add(tf.keras.layers.Dense(units, activation="relu"))
        model.add(tf.keras.layers.Dropout(dropout_rate))

    # Output layer
    model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

    # Compile
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )
    return model

def build_conv1d(seq_len, num_features, config="medium"):
    """
    Build Conv1D model for time series
    """
    if config == "small":
        filters = [32]
        kernel_size = 3
        dense_units = [32]
        dropout_rate = 0.3

    elif config == "medium":
        filters = [64, 32]
        kernel_size = 3
        dense_units = [64, 32]
        dropout_rate = 0.3

    elif config == "large":
        filters = [128, 64]
        kernel_size = 5
        dense_units = [128, 64, 32]
        dropout_rate = 0.4

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=(seq_len, num_features)))

    # Conv1D layers
    for i, f in enumerate(filters):
        model.add(tf.keras.layers.Conv1D(filters=f, kernel_size=kernel_size, activation="relu"))
        model.add(tf.keras.layers.Dropout(dropout_rate))

    model.add(tf.keras.layers.Flatten())

    # Dense layers
    for units in dense_units:
        model.add(tf.keras.layers.Dense(units, activation="relu"))
        model.add(tf.keras.layers.Dropout(dropout_rate))

    # Output layer
    model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

    # Compile
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )
    return model

In [4]:
# ----1. Build Sequences -----------
TOP_FEATURES = ['volatility_21d', 'volatility_10d', 'return_14d', 'return_3d', 'bollinger_down']

def build_sequences(df, seq_len=20, target_col="target"):
    """
    Build sequences but only with top selected features
    """
    values = df[TOP_FEATURES].values  # Use only top features
    targets = df[target_col].values

    X, y = [], []
    for i in range(len(df) - seq_len):
        X.append(values[i:i+seq_len])
        y.append(targets[i+seq_len])
    return np.array(X), np.array(y)

# ----2. Get Callbacks -----------
def get_callbacks(output_dir="experiments", model_name="model"):

    os.makedirs(output_dir, exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    run_dir = os.path.join(output_dir, f"{model_name}_{timestamp}")
    os.makedirs(run_dir, exist_ok=True)

    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor="val_loss",
            patience=5,
            restore_best_weights=True,
            verbose=1
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss",
            factor=0.5,
            patience=3,
            min_lr=1e-6,
            verbose=1
        ),
        tf.keras.callbacks.ModelCheckpoint(
            filepath=os.path.join(run_dir, "best_model.h5"),
            monitor="val_loss",
            save_best_only=True,
            verbose=1
        ),
        tf.keras.callbacks.CSVLogger(
            filename=os.path.join(run_dir, "training_log.csv")
        )
    ]

    return callbacks, run_dir

In [5]:
# Generate OOF predictions
def oof_predictions(X, y, build_fn, model_name):
    tscv = TimeSeriesSplit(n_splits=N_SPLITS)
    oof = np.zeros(len(X))
    model_dir = os.path.join(EXPERIMENTS_DIR, model_name)
    os.makedirs(model_dir, exist_ok=True)

    fold = 0
    for train_idx, val_idx in tscv.split(X):
        fold += 1
        print(f"\n Training {model_name} Fold {fold}/{N_SPLITS}")

        X_tr, X_val = X[train_idx], X[val_idx]
        y_tr, y_val = y[train_idx], y[val_idx]

        # ---- 1. Compute class weights for this fold ----
        classes = np.unique(y_tr)  # Use only training labels for this fold
        weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_tr)
        class_weights = dict(zip(classes, weights))
        
        model = build_fn(SEQ_LEN, X.shape[2])
        callbacks, run_dir = get_callbacks(output_dir=model_dir, model_name=f"{model_name}_fold{fold}")
        
        model.fit(
            tf.data.Dataset.from_tensor_slices((X_tr, y_tr)).shuffle(1000).batch(BATCH_SIZE),
            validation_data=tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(BATCH_SIZE),
            epochs=EPOCHS,
            callbacks=callbacks,
            verbose=1,
            class_weight=class_weights
        )

        # load best weights if saved
        best_path = os.path.join(run_dir, "best_model.h5")
        if os.path.exists(best_path):
            model.load_weights(best_path)

        preds = model.predict(X_val).ravel()
        oof[val_idx] = preds

        # save fold model
        model.save(os.path.join(model_dir, f"{model_name}_fold{fold}.h5"))

    np.save(os.path.join(model_dir, f"{model_name}_oof.npy"), oof)
    return oof

def find_best_threshold(y_true, y_prob):
    thresholds = np.linspace(0.1, 0.9, 81)  # step size = 0.01
    best_thr, best_f1 = 0.5, 0
    for thr in thresholds:
        f1 = f1_score(y_true, (y_prob > thr).astype(int))
        if f1 > best_f1:
            best_f1, best_thr = f1, thr
    return best_thr, best_f1

def ensemble():
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    run_dir = os.path.join(EXPERIMENTS_DIR, f"ensemble_{timestamp}")
    os.makedirs(run_dir, exist_ok=True)

    if not os.path.exists(INPUT_FILE):
        raise FileNotFoundError("Run make_labels.py first to generate labeled dataset")

    df = pd.read_parquet(INPUT_FILE)
    look_ahead = 3  # days
    threshold = 0.01  # +1% move

    df['future_return'] = df['close'].shift(-look_ahead) / df['close'] - 1
    df['target'] = (df['future_return'] > threshold).astype(int)
    X, y = build_sequences(df, seq_len=SEQ_LEN)
    print("Data ready:", X.shape, y.shape)

    # Base model OOF predictions
    oof_lstm = oof_predictions(X, y, build_lstm, "lstm")
    oof_gru = oof_predictions(X, y, build_gru, "gru")
    oof_conv = oof_predictions(X, y, build_conv1d, "conv1d")

    # Save stacking dataset
    stack_df = pd.DataFrame({
        "prob_lstm": oof_lstm,
        "prob_gru": oof_gru,
        "prob_conv": oof_conv,
        "target": y
    })
    stack_file = os.path.join(run_dir, "stacking_oof.csv")
    stack_df.to_csv(stack_file, index=False)
    print(f"Stacking dataset saved to {stack_file}")

    # Averaging ensemble evaluation
    avg_prob = stack_df[["prob_lstm", "prob_gru", "prob_conv"]].mean(axis=1).values
    avg_auc = roc_auc_score(y, avg_prob)
    avg_acc = accuracy_score(y, (avg_prob > 0.5).astype(int))
    avg_f1 = f1_score(y, (avg_prob > 0.5).astype(int))
    print(f"\nAveraging Ensemble - AUC: {avg_auc:.4f}, Acc: {avg_acc:.4f}, F1: {avg_f1:.4f}")

    # Stacking ensemble (meta-learner)
    X_stack = stack_df[["prob_lstm", "prob_gru", "prob_conv"]].values
    y_stack = stack_df["target"].values

    scaler = StandardScaler()
    X_stack = scaler.fit_transform(X_stack)
   
    tscv_meta = TimeSeriesSplit(n_splits=3)
    meta_oof = np.zeros(len(X_stack))  # store all meta predictions

    for fold, (train_idx, val_idx) in enumerate(tscv_meta.split(X_stack)):
        print(f"Meta fold {fold+1}: train {len(train_idx)}, val {len(val_idx)}")
        X_train, X_hold = X_stack[train_idx], X_stack[val_idx]
        y_train, y_hold = y_stack[train_idx], y_stack[val_idx]

        meta = XGBClassifier(
            n_estimators=100,
            max_depth=3,
            learning_rate=0.05,
            subsample=0.8,
            colsample_bytree=0.8,
            reg_alpha=0.5,
            reg_lambda=1.0,
            random_state=42,
            use_label_encoder=False,
            eval_metric="logloss"
        )
        meta.fit(X_train, y_train)
        meta_oof[val_idx] = meta.predict_proba(X_hold)[:, 1]

    # # --- Optimize Threshold ---
    best_thr, best_f1 = find_best_threshold(y_stack, meta_oof)
    best_thr = 0.5
    best_thr, best_f1 = find_best_threshold(y, avg_prob)
    print(f"\nOptimal threshold for F1: {best_thr:.2f} (F1={best_f1:.4f})")

    stack_auc = roc_auc_score(y_stack, meta_oof)
    stack_acc = accuracy_score(y_stack, (meta_oof > best_thr).astype(int))
    stack_f1 = f1_score(y_stack, (meta_oof > best_thr).astype(int))
    print(f"\nStacking Ensemble - AUC: {stack_auc:.4f}, Acc: {stack_acc:.4f}, F1: {stack_f1:.4f}")

    # Save meta-learner
    meta_file = os.path.join(run_dir, "meta_model.pkl")
    joblib.dump(meta, meta_file)
    print(f"Meta-learner saved to {meta_file}")

    # # Compare base vs ensembles
    print("\nComparison of Models (using OOF predictions):")
    print(f"Note: Stacking Ensemble uses optimized threshold = {best_thr:.2f}")

    print(stack_df[["prob_lstm", "prob_gru", "prob_conv"]].corr())

    for name, preds in {
        "LSTM": oof_lstm,
        "GRU": oof_gru,
        "Conv1D": oof_conv,
        "Averaging Ensemble": avg_prob,
        "Stacking Ensemble": meta_oof  # full OOF for meta
    }.items():
        y_true = y  # use full target for all models
        threshold = best_thr if name == "Averaging Ensemble" else 0.5
        y_pred_binary = (preds > threshold).astype(int)

        auc = roc_auc_score(y_true, preds)
        acc = accuracy_score(y_true, y_pred_binary)
        f1 = f1_score(y_true, y_pred_binary)
        print(f"{name:20s}  AUC: {auc:.4f}  Acc: {acc:.4f}  F1: {f1:.4f}")

ensemble()

Data ready: (3689, 20, 5) (3689,)

 Training lstm Fold 1/7
Epoch 1/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.7009 - loss: 0.5997
Epoch 1: val_loss improved from inf to 0.39388, saving model to experiments\lstm\lstm_fold1_20250928-203108\best_model.h5


15/15 ━━━━━━━━━━━━━━━━━━━━ 10s 123ms/step - accuracy: 0.7117 - loss: 0.5924 - val_accuracy: 0.8677 - val_loss: 0.3939 - learning_rate: 0.0010
Epoch 2/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 1.0000 - loss: 0.1278
Epoch 2: val_loss did not improve from 0.39388
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step - accuracy: 1.0000 - loss: 0.1253 - val_accuracy: 0.8677 - val_loss: 0.5298 - learning_rate: 0.0010
Epoch 3/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 1.0000 - loss: 0.0173
Epoch 3: val_loss did not improve from 0.39388
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - accuracy: 1.0000 - loss: 0.0170 - val_accuracy: 0.8677 - val_loss: 0.8279 - learning_rate: 0.0010
Epoch 4/30
14/15 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 1.0000 - loss: 0.0067
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 4: val_loss did not improve from 0.39388
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - accuracy: 1.0000 - loss: 0.0064 - val_accuracy: 0.8677 - val_lo


 Training lstm Fold 2/7
Epoch 1/30
27/29 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6222 - loss: 0.6536
Epoch 1: val_loss improved from inf to 0.60773, saving model to experiments\lstm\lstm_fold2_20250928-203125\best_model.h5


29/29 ━━━━━━━━━━━━━━━━━━━━ 6s 37ms/step - accuracy: 0.6217 - loss: 0.6487 - val_accuracy: 0.6421 - val_loss: 0.6077 - learning_rate: 0.0010
Epoch 2/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.6511 - loss: 0.4670
Epoch 2: val_loss improved from 0.60773 to 0.59580, saving model to experiments\lstm\lstm_fold2_20250928-203125\best_model.h5


29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.6518 - loss: 0.4677 - val_accuracy: 0.6768 - val_loss: 0.5958 - learning_rate: 0.0010
Epoch 3/30
28/29 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.6957 - loss: 0.4718
Epoch 3: val_loss improved from 0.59580 to 0.57269, saving model to experiments\lstm\lstm_fold2_20250928-203125\best_model.h5


...

101/101 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7694 - loss: 0.4333 - val_accuracy: 0.4729 - val_loss: 0.7249 - learning_rate: 0.0010
Epoch 8/30
 98/101 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7562 - loss: 0.4355
Epoch 8: val_loss did not improve from 0.72485
101/101 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7519 - loss: 0.4402 - val_accuracy: 0.4729 - val_loss: 0.7421 - learning_rate: 0.0010
Epoch 9/30
 90/101 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7520 - loss: 0.4269
Epoch 9: val_loss improved from 0.72485 to 0.71171, saving model to experiments\conv1d\conv1d_fold7_20250928-203559\best_model.h5


101/101 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7402 - loss: 0.4409 - val_accuracy: 0.4729 - val_loss: 0.7117 - learning_rate: 0.0010
Epoch 10/30
 95/101 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7481 - loss: 0.4313
Epoch 10: val_loss did not improve from 0.71171
101/101 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7410 - loss: 0.4393 - val_accuracy: 0.4729 - val_loss: 0.7358 - learning_rate: 0.0010
Epoch 11/30
 97/101 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7541 - loss: 0.4238
Epoch 11: val_loss did not improve from 0.71171
101/101 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7492 - loss: 0.4295 - val_accuracy: 0.4729 - val_loss: 0.7484 - learning_rate: 0.0010
Epoch 12/30
 95/101 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7646 - loss: 0.4237
Epoch 12: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 12: val_loss did not improve from 0.71171
101/101 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7576 - loss: 0.4315 - val_accuracy: 0.4

Stacking dataset saved to experiments\ensemble_20250928-203107\stacking_oof.csv

Averaging Ensemble - AUC: 0.7182, Acc: 0.6414, F1: 0.5664
Meta fold 1: train 923, val 922


c:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [20:36:12] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Meta fold 2: train 1845, val 922
Meta fold 3: train 2767, val 922

Optimal threshold for F1: 0.43 (F1=0.5779)

Stacking Ensemble - AUC: 0.6464, Acc: 0.6099, F1: 0.3879
Meta-learner saved to experiments\ensemble_20250928-203107\meta_model.pkl

Comparison of Models (using OOF predictions):
Note: Stacking Ensemble uses optimized threshold = 0.43
           prob_lstm  prob_gru  prob_conv
prob_lstm   1.000000  0.925605   0.905651
prob_gru    0.925605  1.000000   0.954120
prob_conv   0.905651  0.954120   1.000000
LSTM                  AUC: 0.7158  Acc: 0.6552  F1: 0.5411
GRU                   AUC: 0.7155  Acc: 0.6449  F1: 0.5429
Conv1D                AUC: 0.7114  Acc: 0.6349  F1: 0.5787
Averaging Ensemble    AUC: 0.7182  Acc: 0.6246  F1: 0.5779
Stacking Ensemble     AUC: 0.6464  Acc: 0.6256  F1: 0.2743


In [6]:
# # CONFIG
# EXPERIMENTS_DIR = "experiments"
# ensemble_dirs = glob.glob(os.path.join(EXPERIMENTS_DIR, "ensemble_*"))
# latest_ensemble_dir = max(ensemble_dirs, key=os.path.getmtime)
# STACK_PATH = os.path.join(latest_ensemble_dir, "stacking_oof.csv")
# print("Using stacking file:", STACK_PATH)
# OUTPUT_DIR = os.path.join(EXPERIMENTS_DIR, "calibration")
# os.makedirs(OUTPUT_DIR, exist_ok=True)

# PLATT_FILE = os.path.join(OUTPUT_DIR, "calibrator_platt.pkl")
# ISO_FILE = os.path.join(OUTPUT_DIR, "calibrator_isotonic.pkl")
# REPORT_FILE = os.path.join(OUTPUT_DIR, "calibration_report.json")

# def load_stack(stack_path=STACK_PATH):
#     if not os.path.exists(stack_path):
#         raise FileNotFoundError(f"Stacking file not found at {stack_path}")
#     df = pd.read_csv(stack_path)
#     # Expect columns: prob_lstm, prob_gru, prob_conv, target OR 'avg_prob' if present
#     if "avg_prob" in df.columns:
#         avg = df["avg_prob"].values
#     else:
#         avg = df[["prob_lstm", "prob_gru", "prob_conv"]].mean(axis=1).values
#     y = df["target"].values
#     return avg, y, df

# def train_calibrators(probs, y, holdout_frac=0.2, random_state=42):
#     """
#     Train Platt (logistic) and Isotonic calibrators on OOF probs.
#     Use time-aware split if your data is temporal: here we perform a simple holdout
#     that preserves order by splitting by index (no shuffle).
#     """
#     n = len(probs)
#     split_idx = int(n * (1 - holdout_frac))

#     # Time-preserving split (do not shuffle) - use first portion for train, last for holdout
#     p_train, p_hold = probs[:split_idx], probs[split_idx:]
#     y_train, y_hold = y[:split_idx], y[split_idx:]

#     # --- Platt (LogisticRegression on single-column probs) ---
#     platt = LogisticRegression(max_iter=1000)
#     platt.fit(p_train.reshape(-1, 1), y_train)

#     # --- Isotonic ---
#     iso = IsotonicRegression(out_of_bounds="clip")
#     iso.fit(p_train, y_train)

#     # Evaluate on holdout
#     results = {}
#     for name, calib in [("platt", platt), ("isotonic", iso)]:
#         if name == "platt":
#             p_cal = platt.predict_proba(p_hold.reshape(-1, 1))[:, 1]
#         else:
#             p_cal = iso.predict(p_hold)

#         p_cal = np.clip(p_cal, 1e-15, 1 - 1e-15)  # avoid log(0)
#         results[name] = {
#             "brier": float(brier_score_loss(y_hold, p_cal)),
#             "logloss": float(log_loss(y_hold, p_cal)),
#             "roc_auc": float(roc_auc_score(y_hold, p_cal))
#         }

#     # Save calibrators and holdout split indices for traceability
#     joblib.dump(platt, PLATT_FILE)
#     joblib.dump(iso, ISO_FILE)

#     report = {
#         "n_samples": int(n),
#         "holdout_start_index": int(split_idx),
#         "platt_path": PLATT_FILE,
#         "isotonic_path": ISO_FILE,
#         "metrics_holdout": results
#     }
#     with open(REPORT_FILE, "w") as f:
#         json.dump(report, f, indent=2)

#     return platt, iso, report

# def apply_calibrator(probs, method="platt"):
#     """
#     Apply saved calibrator to a numpy array of probs (new/test).
#     method: "platt" or "isotonic"
#     """
#     if method == "platt":
#         if not os.path.exists(PLATT_FILE):
#             raise FileNotFoundError("Platt calibrator not found. Run training first.")
#         platt = joblib.load(PLATT_FILE)
#         return platt.predict_proba(np.array(probs).reshape(-1, 1))[:, 1]
#     elif method == "isotonic":
#         if not os.path.exists(ISO_FILE):
#             raise FileNotFoundError("Isotonic calibrator not found. Run training first.")
#         iso = joblib.load(ISO_FILE)
#         return iso.predict(np.array(probs))
#     else:
#         raise ValueError("method must be 'platt' or 'isotonic'")

# def summary_print(report):
#     print("Calibration report summary")
#     print(f"Samples: {report['n_samples']}, holdout starts at index {report['holdout_start_index']}")
#     for name, m in report["metrics_holdout"].items():
#         print(f"{name.upper():8s}  Brier: {m['brier']:.5f}  LogLoss: {m['logloss']:.5f}  ROC-AUC: {m['roc_auc']:.5f}")
#     print("Saved calibrators to:", report["platt_path"], report["isotonic_path"])
#     print("Full report JSON:", REPORT_FILE)


# # Load OOF average probs and targets
# probs, y, df = load_stack(STACK_PATH)

# # Train calibrators and save
# platt, iso, report = train_calibrators(probs, y, holdout_frac=0.2)
# summary_print(report)

# # Example: apply calibrator to the same OOF holdout set to inspect mapping
# split_idx = report["holdout_start_index"]
# print("Holdout class distribution:")
# print(pd.Series(y[split_idx]).value_counts(normalize=True))

# raw_auc = roc_auc_score(y, probs)
# raw_acc = accuracy_score(y, (probs > 0.5).astype(int))
# raw_f1 = f1_score(y, (probs > 0.5).astype(int))
# print(f"Raw OOF - AUC: {raw_auc:.4f}, Acc: {raw_acc:.4f}, F1: {raw_f1:.4f}")

# p_hold = probs[split_idx:]
# p_platt = apply_calibrator(p_hold, method="platt")
# p_iso = apply_calibrator(p_hold, method="isotonic")

# # Save holdout-calibrated probs for inspection
# hold_df = pd.DataFrame({
#     "raw_prob": p_hold,
#     "platt_prob": p_platt,
#     "isotonic_prob": p_iso,
#     "target": y[split_idx:]
# })
# hold_df.to_csv(os.path.join(OUTPUT_DIR, "holdout_calibrated_probs.csv"), index=False)
# print("Saved holdout calibrated probs to:", os.path.join(OUTPUT_DIR, "holdout_calibrated_probs.csv"))

In [5]:
# How to use in the backtest:

# from calibrate_probs import apply_calibrator

# # suppose avg_prob is your averaged ensemble probabilities (numpy array)
# calibrated_prob = apply_calibrator(avg_prob, method="platt")  # or "isotonic"
# signals = generate_signals(calibrated_prob, threshold=chosen_threshold)

In [7]:
# def ensemble_uncertainty(probs_array):
#     """
#     Compute mean prediction + uncertainty from multiple models.
    
#     Args:
#         probs_array: numpy array of shape (n_models, n_samples)
    
#     Returns:
#         mean_prob: (n_samples,) averaged probability
#         uncertainty: (n_samples,) standard deviation across models
#     """
#     mean_prob = np.mean(probs_array, axis=0)
#     uncertainty = np.std(probs_array, axis=0)
#     return mean_prob, uncertainty


# def mc_dropout_predictions(model, X, n_iter=30, batch_size=32):
#     """
#     Monte Carlo Dropout predictions for uncertainty estimation.
    
#     Args:
#         model: tf.keras.Model with Dropout layers
#         X: input data
#         n_iter: number of stochastic forward passes
#         batch_size: batch size
    
#     Returns:
#         probs: numpy array of shape (n_iter, n_samples)
#     """
#     f = tf.function(lambda inp: model(inp, training=True))
#     preds = []
#     for _ in range(n_iter):
#         p = f(X, training=True).numpy().ravel()
#         preds.append(p)
#     return np.array(preds)


# def mc_dropout_uncertainty(model, X, n_iter=30, batch_size=32):
#     """
#     Mean + uncertainty using MC Dropout.
#     """
#     probs = mc_dropout_predictions(model, X, n_iter=n_iter, batch_size=batch_size)
#     mean_prob = probs.mean(axis=0)
#     uncertainty = probs.std(axis=0)
#     return mean_prob, uncertainty

In [8]:
# # Suppose you have probabilities from your 3 base models
# prob_lstm = np.load("experiments/lstm/lstm_oof.npy")
# prob_gru  = np.load("experiments/gru/gru_oof.npy")
# prob_conv = np.load("experiments/conv1d/conv1d_oof.npy")

# # Stack them
# probs_array = np.vstack([prob_lstm, prob_gru, prob_conv])

# mean_prob, uncertainty = ensemble_uncertainty(probs_array)

# print("First 5 probs:", mean_prob[:5])
# print("First 5 uncertainties:", uncertainty[:5])